In [1]:
import pandas as pd

In [2]:
test_df = pd.read_parquet("data/ml-engineer-main/iot_network_intrustion_dataset_test.parquet")
test_df.head()

,Unnamed: 0,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,230773,163.152.127.193-192.168.0.13-10101-56361-17,192.168.0.13,56361,163.152.127.193,10101,17,20/08/2019 03:05:41 AM,141,2,...,0.0,0.0,0.0,70.5,2.121320,72.0,69.0,Anomaly,Mirai,Mirai-Hostbruteforceg
1,521223,192.168.0.24-210.89.164.90-64776-9988-17,192.168.0.24,64776,210.89.164.90,9988,17,25/07/2019 03:20:13 AM,1,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Anomaly,Mirai,Mirai-UDP Flooding
2,155309,192.168.0.13-192.168.0.16-9020-49784-6,192.168.0.13,9020,192.168.0.16,49784,6,20/05/2019 04:58:07 AM,482,0,...,0.0,0.0,0.0,120.5,96.334487,265.0,72.0,Normal,Normal,Normal
3,42560,192.168.0.24-104.118.134.215-43238-443-6,104.118.134.215,443,192.168.0.24,43238,6,25/07/2019 03:21:48 AM,269,1,...,0.0,0.0,0.0,269.0,0.000000,269.0,269.0,Anomaly,Mirai,Mirai-Ackflooding
4,326612,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:03 AM,73,0,...,0.0,0.0,0.0,73.0,0.000000,73.0,73.0,Anomaly,Scan,Scan Port OS
